# chroma basic

## chroma setup
(already set, so commented out)

In [4]:

import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db") # PersistentClientにすると、DBがfolder保存される


## OpenAI enbedding setup
open ai APIs now do not support multi modal embeddings. 

In [ ]:

# import numpy as np
# import os
# from chromadb.utils import embedding_functions
# from dotenv import load_dotenv
# load_dotenv()

# openai_mm = embedding_functions.OpenAIEmbeddingFunction(
#     api_key = os.environ["OPENAI_API_KEY"],
#     model_name="text-embedding-3-small  mm embedding model"
#     )

(1, 1536)

## OpenCLIP setup

In [2]:
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
embedding_function = OpenCLIPEmbeddingFunction()


c:\Users\ishid\miniconda3\envs\airizap\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\ishid\miniconda3\envs\airizap\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ishid\.cache\huggingface\hub\models--laion--CLIP-ViT-B-32-laion2B-s34B-b79K. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING

## setting collection(=table in RDB)
Chromaフォルダ内でUUIDで識別される

In [ ]:

import chromadb
from chromadb.utils.data_loaders import ImageLoader
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

data_loader = ImageLoader() # OpenCLIPでは直接画像をChroma DBに保存しない。そのため、URI指定で生画像を持ってきてくれるImage Loaderを使用する。
embedding_function = OpenCLIPEmbeddingFunction()

collection = chroma_client.create_collection(
    name='multimodal_collection',
    embedding_function=embedding_function,
    data_loader=data_loader
)

# if you want to make multi modal vector database, you have two options:
# 1. use image -> captions model, then use caption to text embedding model to store chrome db.
# 2. use CLIP(Contrastive Language–Image Pretraining) embedding model to directly embed image and text into same vector space. (to be studied)
# CLIP leasrns a lot of image-text pairs. it is used for image search and classification, generation etc.(like DALL-E, Stable Diffusion etc.)

# 1. を使う場合、ベクトルDBに追加するデータは細切れに、元データは大枠に保存するらしい。ベクター検索は細切れのほうが精度がよく、コンテキストとして使うには大枠のほうがいいため。


for multi modal db, i refered chroma official document: 
[Embeddings - Multimodal](https://docs.trychroma.com/docs/embeddings/multimodal)

## pdf processer

In [ ]:
# to deal with the pdf, you have to;
# - 1. loard pdf,
# - 2. split the pdf into chunks, 
# - 3. embed the chunks, 
# - 4. store the embedded chunks into chroma db.



[REF](https://dev.nagomi-tec.com/archives/2859)

[PDF textスプリットはこれが参考になるかも](https://zenn.dev/ml_bear/books/d1f060a3f166a5/viewer/e1085f)

https://qiita.com/camcam/items/ae9ac4860968389804bd
PDF loader選びが大切そう

## adding document to collection

In [ ]:
# adding image file paths and text documents to collection
collection.add(
    ids=["id1", "id2"],
    uris=["path/to/file/1", "path/to/file/2"]
)

collection.add(
    ids=["id3", "id4"],
    documents=["This is a document", "This is another document"]
)

## seraching DB and result extraction

In [ ]:
# seraching collection
query = "sample"
results = collection.query(
    query_texts=[query], # 検索クエリ(内部的にquery_textをembeddingしてベクトル検索を行う。）
    # query_embeddingsで直接ベクトルを検索をすることもできる。ただし、次元数が合わないとエラーになるなど注意が必要)
    # AZUREopenaiみたいにハイブリッド検索はできない（難しい）けど、無料でここまでできるならすごい
    n_results=3 # 返す件数
    include=["documents","data"] # 返す内容指定. dataを入れるとData loaderが自動的に呼び出され、（画像などの）データも返す。
    # where={} でメタデータ絞り込みも可能
    )
print(results)
print("---------------")
print(results["documents"][0][0])

{'ids': [['doc1']], 'embeddings': None, 'documents': [['This is a sample document.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source': 'sample_source'}]], 'distances': [[0.8754018545150757]]}
---------------
This is a sample document.
